In [1]:
%pip install -q transformers
%pip install  -q sentence-transformers

%pip install -q accelerate
%pip install -q einops
%pip install -q xformers
%pip install -q bitsandbytes

%pip install -q evaluate
%pip install -q bert_score
!pip install -q chromadb
%pip install -q llama-index-vector-stores-chroma
!pip install -q sentence_transformers
!pip install -q llama-index
!pip install -q llama-index-embeddings-langchain
!pip install --quiet langchain_experimental
%pip install -q llama-index-retrievers-bm25
%pip -install -q langchain

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Usage:   
  /home/ubuntu/projects/llmlegalassistant/.venv/bin/python -m pip <command> [options]

no such option: -i
Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging
import sys
from os import getenv
import nest_asyncio
from llama_index.core import Settings



logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext

In [3]:
from torch import cuda, bfloat16
import os
import transformers
from transformers import LlamaTokenizer,AutoTokenizer
hf_auth = os.environ.get('HF_AUTH')
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",use_auth_token=hf_auth)

model_id = 'meta-llama/Llama-2-7b-chat-hf'

bitsAndBites_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=os.environ.get('HF_AUTH')
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bitsAndBites_config,
    device_map='auto',
    token=os.environ.get('HF_AUTH')
)
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    # we pass model parameters here too
    temperature=0.01,
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)

/home/ubuntu/projects/llmlegalassistant/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/projects/llmlegalassistant/.venv/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/ubuntu/projects/llmlegalassistant/.venv/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py:1096: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.90s/it]


In [ ]:
tokenizer.save_pretrained(save_directory="/home/ubuntu/projects/llmlegalassistant/models0/")

In [ ]:
model.save_pretrained(save_directory="/home/ubuntu/projects/llmlegalassistant/models0/")

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained

In [5]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [ ]:

model_name="WhereIsAI/UAE-Large-V1"
device = 'cuda:0'
embed_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [6]:
embed_model_2 = HuggingFaceEmbeddings(model_name="infgrad/stella-base-en-v2")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: infgrad/stella-base-en-v2
Load pretrained SentenceTransformer: infgrad/stella-base-en-v2
No sentence-transformers model found with name infgrad/stella-base-en-v2. Creating a new one with MEAN pooling.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
Use pytorch device_name: cuda


In [7]:
!pip install llama-index-llms-langchain

  Using cached llama_index_llms_langchain-0.1.3-py3-none-any.whl.metadata (801 bytes)
  Using cached llama_index_llms_anyscale-0.1.3-py3-none-any.whl.metadata (698 bytes)
Using cached llama_index_llms_langchain-0.1.3-py3-none-any.whl (4.6 kB)
Using cached llama_index_llms_anyscale-0.1.3-py3-none-any.whl (4.0 kB)


In [8]:
Settings.llm = llm

In [9]:
documents = SimpleDirectoryReader('/home/ubuntu/projects/llmlegalassistant/data_2/data').load_data()

In [10]:
import json
from llama_index.core.llama_dataset import (
    LabeledRagDataExample,
    CreatedByType,
    CreatedBy,
    LabeledRagDataset
)

# Load JSON data
with open('/home/ubuntu/projects/llmlegalassistant/LabelledRAGQAcombined.json', 'r') as f:
    data = json.load(f)

rag_examples = []

# Iterate over examples in the JSON file
for example in data:
    # Extract data from the current example
    question = example['question']
    answer = example['answer']
    context = example['context']

    # Constructing a LabelledRagDataExample
    query = question
    query_by = CreatedBy(type=CreatedByType.HUMAN)
    reference_answer = answer
    reference_answer_by = CreatedBy(type=CreatedByType.HUMAN)
    reference_contexts = [context]

    rag_example = LabeledRagDataExample(
        query=query,
        query_by=query_by,
        reference_contexts=reference_contexts,
        reference_answer=reference_answer,
        reference_answer_by=reference_answer_by,
    )

    # Append the constructed example to the list of rag_examples
    rag_examples.append(rag_example)
rag_dataset = LabeledRagDataset(examples = [x for x in rag_examples])
df = rag_dataset.to_pandas()

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter

splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
)
nodes0 = splitter.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter1 = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=20,
)
nodes1 = splitter.get_nodes_from_documents(documents)

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index.core.node_parser import LangchainNodeParser

In [ ]:


parser = LangchainNodeParser(RecursiveCharacterTextSplitter(chunk_size=512))
nodes2 = parser.get_nodes_from_documents(documents)

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
text_splitter = SemanticChunker(embed_model)
parser = LangchainNodeParser(text_splitter)
nodes3 = parser.get_nodes_from_documents(documents)

In [26]:
from langchain_experimental.text_splitter import SemanticChunker
text_splitter = SemanticChunker(embed_model_2)
parser = LangchainNodeParser(text_splitter)
nodes4 = parser.get_nodes_from_documents(documents)

INFO:langchain_community.utils.math:Unable to import simsimd, defaulting to NumPy implementation. If you want to use simsimd please install with `pip install simsimd`.
Unable to import simsimd, defaulting to NumPy implementation. If you want to use simsimd please install with `pip install simsimd`.
INFO:langchain_community.utils.math:Unable to import simsimd, defaulting to NumPy implementation. If you want to use simsimd please install with `pip install simsimd`.
Unable to import simsimd, defaulting to NumPy implementation. If you want to use simsimd please install with `pip install simsimd`.
INFO:langchain_community.utils.math:Unable to import simsimd, defaulting to NumPy implementation. If you want to use simsimd please install with `pip install simsimd`.
Unable to import simsimd, defaulting to NumPy implementation. If you want to use simsimd please install with `pip install simsimd`.
INFO:langchain_community.utils.math:Unable to import simsimd, defaulting to NumPy implementation. If

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore
docstore = SimpleDocumentStore()
docstore.add_documents(nodes3)
docstore.persist("/home/ubuntu/projects/llmlegalassistant/data_2/docstore/UAE_semantic.json")

In [19]:
from llama_index.core.storage.docstore import SimpleDocumentStore
docstore1 = SimpleDocumentStore.from_persist_path("/home/ubuntu/projects/llmlegalassistant/data_2/docstore/stella_semantic.json")

In [12]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.ingestion import IngestionPipeline
# create client and store
db = chromadb.PersistentClient(path="/home/ubuntu/projects/llmlegalassistant/data_2/Index")

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [ ]:
chroma_collection = db.get_collection("Splitter_text_UAE_emb")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)



storage_context = StorageContext.from_defaults(vector_store=vector_store)
index0 = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context,embed_model = embed_model
)

In [ ]:
# create collection
chroma_collection = db.get_or_create_collection("Spliiter_sentence_UAE_emb")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)



storage_context = StorageContext.from_defaults(vector_store=vector_store)
index1 = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context,embed_model = embed_model
)

In [ ]:
chroma_collection = db.get_or_create_collection("Spliiter_Recursive_UAE_emb")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)



storage_context = StorageContext.from_defaults(vector_store=vector_store)
index2 = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context,embed_model = embed_model
)

In [ ]:
chroma_collection = db.get_or_create_collection("Spliiter_semantic_UAE_emb")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)



storage_context = StorageContext.from_defaults(vector_store=vector_store)
index3 = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context,embed_model = embed_model
)

In [ ]:
chroma_collection = db.get_or_create_collection("Spliiter_text_stella_emb")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)



storage_context = StorageContext.from_defaults(vector_store=vector_store)
index4 = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context,embed_model = embed_model_2
)

In [ ]:
chroma_collection = db.get_or_create_collection("Splitter_sentence_stella_emb")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)



storage_context = StorageContext.from_defaults(vector_store=vector_store)
index5 = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context,embed_model = embed_model_2
)

In [ ]:
chroma_collection = db.get_or_create_collection("Splitter_Recursive_stella_emb")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)



storage_context = StorageContext.from_defaults(vector_store=vector_store)
index6 = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context,embed_model = embed_model_2
)

In [28]:
db.delete_collection('Spliiter_semantic_stella_emb')

In [29]:
chroma_collection = db.get_or_create_collection("Spliiter_semantic_stella_emb")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)



storage_context = StorageContext.from_defaults(vector_store=vector_store)
index7 = VectorStoreIndex(
    nodes =nodes4, storage_context=storage_context,embed_model = embed_model_2
)

In [14]:
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever

In [30]:
vector_retriever = index7.as_retriever(similarity_top_k=1)
nest_asyncio.apply()
from llama_index.core.query_engine import RetrieverQueryEngine

In [ ]:
vector_retriever0 = index0.as_retriever(similarity_top_k=2)
vector_retriever4 = index4.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(nodes= nodes0, similarity_top_k=2) #change nodes for different chunking strategy


retriever_text_UAE = QueryFusionRetriever(
    [vector_retriever0, bm25_retriever],
    similarity_top_k=1,
    num_queries=1,  # setting this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,

)
retriever_text_stella = QueryFusionRetriever(
    [vector_retriever4, bm25_retriever],
    similarity_top_k=1,
    num_queries=1,  # setting this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,

)


In [ ]:
vector_retriever1 = index1.as_retriever(similarity_top_k=2)
vector_retriever5 = index5.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(nodes= nodes1, similarity_top_k=2) #change nodes for different chunking strategy


retriever_sentence_UAE = QueryFusionRetriever(
    [vector_retriever1, bm25_retriever],
    similarity_top_k=1,
    num_queries=1,  # setting this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,

)
retriever_sentence_stella = QueryFusionRetriever(
    [vector_retriever5, bm25_retriever],
    similarity_top_k=1,
    num_queries=1,  # setting this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,

)

In [ ]:
vector_retriever2 = index2.as_retriever(similarity_top_k=2)
vector_retriever6 = index6.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(nodes= nodes2, similarity_top_k=2) #change nodes for different chunking strategy


retriever_Recursive_UAE = QueryFusionRetriever(
    [vector_retriever2, bm25_retriever],
    similarity_top_k=1,
    num_queries=1,  # setting this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,

)
retriever_Recursive_stella = QueryFusionRetriever(
    [vector_retriever6, bm25_retriever],
    similarity_top_k=1,
    num_queries=1,  # setting this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,

)

In [ ]:
vector_retriever3 = index3.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(nodes= nodes3, similarity_top_k=2
)                                 #change nodes for different chunking strategy



retriever_semantic_UAE = QueryFusionRetriever(
    [vector_retriever3, bm25_retriever],
    similarity_top_k=1,
    num_queries=1,  # setting this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,

)

In [ ]:
vector_retriever7 = index7.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(nodes= nodes4, similarity_top_k=2
)                                 #change nodes for different chunking strategy

from llama_index.core.retrievers import QueryFusionRetriever

retriever_semantic_stella = QueryFusionRetriever(
    [vector_retriever7, bm25_retriever],
    similarity_top_k=1,
    num_queries=1,  # setting this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,

)

In [ ]:
nest_asyncio.apply()
from llama_index.core.query_engine import RetrieverQueryEngine

In [ ]:
query_engine_fusion = RetrieverQueryEngine.from_args(retriever_text_UAE)
for query in df['query']:
    answer = query_engine_fusion.query(query)

    # Creating a dictionary to hold the response and context
    data = {
        "query":query,
        "geneated_response": answer.response,
        "retrieved_context": answer.source_nodes[0].text
    }

    # Appending the data to a JSON file
    with open('/home/ubuntu/projects/llmlegalassistant/responses_Splitter_text_UAE_emb_fusion.json', 'a') as json_file:
        json.dump(data, json_file)
        json_file.write('\n')

In [ ]:
query_engine_fusion = RetrieverQueryEngine.from_args(retriever_text_stella)
for query in df['query'].head(50):
    answer = query_engine_fusion.query(query)

    # Creating a dictionary to hold the response and context
    data = {
        "query":query,
        "geneated_response": answer.response,
        "retrieved_context": answer.source_nodes[0].text
    }

    # Appending the data to a JSON file
    with open('/home/ubuntu/projects/llmlegalassistant/responses_Splitter_text_stella_emb_fusion.json', 'a') as json_file:
        json.dump(data, json_file)

In [ ]:
query_engine_fusion = RetrieverQueryEngine.from_args(retriever_sentence_UAE)
for query in df['query']:
    answer = query_engine_fusion.query(query)

    # Creating a dictionary to hold the response and context
    data = {
        "query":query,
        "geneated_response": answer.response,
        "retrieved_context": answer.source_nodes[0].text
    }

    # Appending the data to a JSON file
    with open('/home/ubuntu/projects/llmlegalassistant/responses_Splitter_sentence_UAE_emb_fusion.json', 'a') as json_file:
        json.dump(data, json_file)
        json_file.write('\n')

In [ ]:
query_engine_fusion = RetrieverQueryEngine.from_args(retriever_sentence_stella)
for query in df['query']:
    answer = query_engine_fusion.query(query)

    # Creating a dictionary to hold the response and context
    data = {
        "query":query,
        "geneated_response": answer.response,
        "retrieved_context": answer.source_nodes[0].text
    }

    # Appending the data to a JSON file
    with open('/home/ubuntu/projects/llmlegalassistant/responses_Splitter_sentence_stella_emb_fusion.json', 'a') as json_file:
        json.dump(data, json_file)
        json_file.write('\n')

In [ ]:
query_engine_fusion = RetrieverQueryEngine.from_args(retriever_Recursive_UAE)
for query in df['query']:
    answer = query_engine_fusion.query(query)

    # Creating a dictionary to hold the response and context
    data = {
        "query":query,
        "geneated_response": answer.response,
        "retrieved_context": answer.source_nodes[0].text
    }

    # Appending the data to a JSON file
    with open('/home/ubuntu/projects/llmlegalassistant/responses_Splitter_Recursive_UAE_emb_fusion.json', 'a') as json_file:
        json.dump(data, json_file)
        json_file.write('\n')

In [ ]:
query_engine_fusion = RetrieverQueryEngine.from_args(retriever_Recursive_stella)
for query in df['query']:
    answer = query_engine_fusion.query(query)

    # Creating a dictionary to hold the response and context
    data = {
        "query":query,
        "geneated_response": answer.response,
        "retrieved_context": answer.source_nodes[0].text
    }

    # Appending the data to a JSON file
    with open('/home/ubuntu/projects/llmlegalassistant/responses_Splitter_Recursive_stella_emb_fusion.json', 'a') as json_file:
        json.dump(data, json_file)
        json_file.write('\n')

In [ ]:
query_engine_fusion = RetrieverQueryEngine.from_args(retriever_semantic_UAE)
data_list = []
for query in df['query'].head(50):
    answer = query_engine_fusion.query(query)

    # Creating a dictionary to hold the response and context
    data = {
        "query":query,
        "geneated_response": answer.response,
        "retrieved_context": answer.source_nodes[0].text
    }

    data_list.append(data)

    # Appending the data to a JSON file
    with open('/home/ubuntu/projects/llmlegalassistant/responses_Splitter_semantic_UAE_emb_fusion.jsonl', 'w') as json_file:
        json.dump(data_list, json_file)

In [18]:
index7.summary

'None'

In [31]:
query_engine_fusion = RetrieverQueryEngine.from_args(vector_retriever)
data_list = []
for query in df['query'].head(50):
    answer = query_engine_fusion.query(query)

    # Creating a dictionary to hold the response and context
    data = {
        "query":query,
        "generated_response": answer.response,
        "retrieved_context": answer.source_nodes[0].text
    }

    data_list.append(data)

    # Appending the data to a JSON file
    with open('/home/ubuntu/projects/llmlegalassistant/responses_Splitter_semantic_stella_emb_vector.jsonl', 'w') as json_file:
        json.dump(data_list, json_file)

/home/ubuntu/projects/llmlegalassistant/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/ubuntu/projects/llmlegalassistant/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/ubuntu/projects/llmlegalassistant/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/ubuntu/projects/llmlegalassistant/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a da

In [ ]:
bm25_retriever = BM25Retriever.from_defaults(nodes= nodes0, similarity_top_k=1)

In [ ]:
query_engine_fusion = RetrieverQueryEngine.from_args(bm25_retriever)
data_list = []
for query in df['query'].head(50):
    answer = query_engine_fusion.query(query)

    # Creating a dictionary to hold the response and context
    data = {
        "query":query,
        "geneated_response": answer.response,
        "retrieved_context": answer.source_nodes[0].text
    }

    data_list.append(data)

    # Appending the data to a JSON file
    with open('/home/ubuntu/projects/llmlegalassistant/responses_Splitter_text_bm25.jsonl', 'w') as json_file:
        json.dump(data_list, json_file)